# Input Features as Output Objectives

This notebook demonstrates how to put objectives on input features or a combination of input features. Possible usecases are favoring lower or higher amounts of an ingredient or to take into account a known (linear) cost function. In case of categorical inputs it can be used to penalize the optimizer for choosing specific categories.

## Imports

In [ ]:
import numpy as np

import bofire.strategies.api as strategies
import bofire.surrogates.api as surrogates
from bofire.benchmarks.api import Himmelblau
from bofire.data_models.features.api import CategoricalInput, ContinuousOutput
from bofire.data_models.objectives.api import (
    MaximizeObjective,
    MaximizeSigmoidObjective,
)
from bofire.data_models.strategies.api import MultiplicativeSoboStrategy
from bofire.data_models.surrogates.api import (
    BotorchSurrogates,
    CategoricalDeterministicSurrogate,
    LinearDeterministicSurrogate,
)

## Setup an Example

We use Himmelblau as example with an additional objective on `x_2` which pushes it to be larger 3 during the optimization. In addition, we introduce a categorical feature called `x_cat` which is mapped by an `CategoricalDeterministicSurrogate` to a continuous output called `y_cat`.

In [ ]:
bench = Himmelblau()
experiments = bench.f(bench.domain.inputs.sample(10), return_complete=True)

domain = bench.domain

# setup extra feature `y_x2` that is the same as `x_2` and is taken into account in the optimization by a sigmoid objective
domain.outputs.features.append(
    ContinuousOutput(key="y_x2", objective=MaximizeSigmoidObjective(tp=3, steepness=10))
)
experiments["y_x2"] = experiments.x_2


# add extra categorical input feature and corresponding output feature
domain.inputs.features.append(CategoricalInput(key="x_cat", categories=["a", "b", "c"]))
domain.outputs.features.append(
    ContinuousOutput(key="y_cat", objective=MaximizeObjective())
)

# generate random values for the new categorical feature
experiments["x_cat"] = np.random.choice(["a", "b", "c"], size=experiments.shape[0])

The `LinearDeterministicSurrogate` can be used to model that `y_x2 = x_2`.

In [ ]:
surrogate_data = LinearDeterministicSurrogate(
    inputs=domain.inputs.get_by_keys(["x_2"]),
    outputs=domain.outputs.get_by_keys(["y_x2"]),
    coefficients={"x_2": 1},
    intercept=0,
)
surrogate = surrogates.map(surrogate_data)
surrogate.predict(experiments[domain.inputs.get_keys()].copy())

,y_x2_pred,y_x2_sd
0,-4.024074,0.0
1,-0.227390,0.0
2,0.942728,0.0
3,4.952023,0.0
4,-2.740823,0.0
5,-2.907850,0.0
6,4.664849,0.0
7,1.925072,0.0
8,5.635899,0.0
9,-3.496666,0.0


The `CategoricalDeterministicSurrogate` can be used to map categories to specific continuous values.

In [ ]:
categorical_surrogate_data = CategoricalDeterministicSurrogate(
    inputs=domain.inputs.get_by_keys(["x_cat"]),
    outputs=domain.outputs.get_by_keys(["y_cat"]),
    mapping={"a": 1, "b": 0.2, "c": 0.3},
)

surrogate = surrogates.map(categorical_surrogate_data)

surrogate.predict(experiments[domain.inputs.get_keys()].copy())

experiments["y_cat"] = surrogate.predict(experiments[domain.inputs.get_keys()].copy())[
    "y_cat_pred"
]

experiments

,x_1,x_2,y,valid_y,y_x2,x_cat,y_cat
0,4.011654,5.477016,841.238569,1,5.477016,b,0.2
1,0.005947,3.827489,110.052946,1,3.827489,b,0.2
2,4.445083,5.787782,1169.102059,1,5.787782,b,0.2
3,-4.808921,4.621849,371.732653,1,4.621849,c,0.3
4,-3.781533,2.441688,56.196349,1,2.441688,b,0.2
5,-5.818184,0.002023,686.578453,1,0.002023,a,1.0
6,-1.685204,1.349516,93.498541,1,1.349516,c,0.3
7,-4.948439,-2.003451,194.831212,1,-2.003451,b,0.2
8,-3.901689,2.982128,55.950861,1,2.982128,c,0.3
9,2.079300,5.570553,682.973914,1,5.570553,a,1.0


Next we setup a `SoboStrategy` using the custom surrogates for outputs `y_x2` and `y_cat` and ask for a candidate. Note that the surrogate specs for output `y` is automatically generated and defaulted to be a `SingleTaskGPSurrogate`.

In [ ]:
strategy_data = MultiplicativeSoboStrategy(
    domain=domain,
    surrogate_specs=BotorchSurrogates(
        surrogates=[surrogate_data, categorical_surrogate_data]
    ),
)
strategy = strategies.map(strategy_data)
strategy.tell(experiments)
strategy.ask(4)

/Users/j30607/sandbox/botorch/botorch/models/ensemble.py:69: RuntimeWarning: Could not update `train_inputs` with transformed inputs since AffineDeterministicModel does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  self.eval()
/opt/homebrew/Caskroom/miniforge/base/envs/bofire-2/lib/python3.10/site-packages/torch/nn/modules/module.py:2417: RuntimeWarning: Could not update `train_inputs` with transformed inputs since AffineDeterministicModel does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  return self.train(False)
/Users/j30607/sandbox/botorch/botorch/models/ensemble.py:69: RuntimeWarning: Could not update `train_inputs` with transformed inputs since AffineDeterministicModel does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  self.eval()
/opt

,x_1,x_2,x_cat,y_pred,y_cat_pred,y_x2_pred,y_sd,y_cat_sd,y_x2_sd,y_des,y_x2_des,y_cat_des
0,-1.099310,5.815160,a,148.146187,1.0,5.815160,290.585241,0.0,0.0,-148.146187,1.000000,1.0
1,-2.804909,5.142759,a,166.904390,1.0,5.142759,296.498962,0.0,0.0,-166.904390,1.000000,1.0
2,-0.779450,3.374349,a,75.545561,1.0,3.374349,204.728780,0.0,0.0,-75.545561,0.976876,1.0
3,-2.048194,6.000000,a,196.116183,1.0,6.000000,316.750102,0.0,0.0,-196.116183,1.000000,1.0
